In [ ]:
#Kirkenes airport and the smaller airports along the coast of Finnmark, like Vadsø, Vardø, Båtsfjord, Berlevåg and Mehamn.

In [2]:
%pip install unidecode


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


https://www.thebarentsobserver.com/security/russian-jamming-is-now-messing-up-gps-signals-for-norwegian-aviation-practically-every-day/163663
https://www.thebarentsobserver.com/security/intensified-gps-jamming-is-side-effect-of-russias-selfprotection-of-kola-bases/166119
https://www.thebarentsobserver.com/news/we-were-spoofed-on-approaching-kirkenes-today/423323


In [7]:
import pandas as pd
#import unidecode

In [9]:
airports_df=pd.read_csv(r'https://github.com/jpatokal/openflights/blob/master/data/airports.dat',na_values=r'\N')

ParserError: Error tokenizing data. C error: Expected 1 fields in line 42, saw 40
